<a href="https://colab.research.google.com/github/fpcaforio/grace/blob/master/gradcam%2BkMeans%2BkNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLS-KDD

In [1]:
#import

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
from keras.models import load_model

import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from PIL import Image
import pickle

from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D
from keras.models import Model

from tensorflow.keras import backend as K
#tf.compat.v1.disable_eager_execution()
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D
from sklearn.metrics import confusion_matrix


import re
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.image as mpimg
import keras.backend as K

from keras.preprocessing import image
from keras.layers import Dense, Input, Layer, InputSpec, Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, normalized_mutual_info_score, adjusted_rand_score#, confusion_matrix
from sklearn.utils.linear_assignment_ import linear_assignment

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

from keras import callbacks
from sklearn.model_selection import train_test_split


import time

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/gradcam/pickle/testing/XTrain_heatmap_12x12.pickle', 'rb')
XTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/MAGNETO/12x12/Ytrain.pickle', 'rb')
YTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/gradcam/pickle/testing/XTest_heatmap_12x12.pickle', 'rb')
XTest = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/MAGNETO/12x12/Ytest.pickle', 'rb')
YTest = pickle.load(f_myfile)
f_myfile.close()

XTrain = np.array(XTrain)
XTrain = np.uint8(255 * XTrain)
XTrain = XTrain / 255

XTest = np.array(XTest)
XTest = np.uint8(255 * XTest)
XTest = XTest / 255

#plt.imshow(XTrain[0])

XTrain = XTrain.reshape(-1, (12*12))
XTest = XTest.reshape(-1, (12*12))

print(XTrain.shape)
print(XTest.shape)

(125973, 144)
(22544, 144)


In [ ]:
'''
img = XTrain[0].reshape(12,12)
plt.imshow(img)
'''

In [4]:
XTrain_normal = XTrain[np.where(YTrain[:] == 1)]
XTrain_attack = XTrain[np.where(YTrain[:] == 0)]

print('Train')
print('Train normal: ', XTrain_normal.shape)
print('Train attack: ', XTrain_attack.shape)

Train
Train normal:  (67343, 144)
Train attack:  (58630, 144)


In [6]:
#sklearn.cluster.KMeans

from sklearn.cluster import KMeans
import numpy as np
import time

start_time = time.time()  #time
kmeans_normal = KMeans(n_clusters=1000, random_state=0).fit(XTrain_normal)
centroids_normal = kmeans_normal.cluster_centers_
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time
kmeans_attack = KMeans(n_clusters=1000, random_state=0).fit(XTrain_attack)
centroids_attack = kmeans_attack.cluster_centers_
print(centroids_attack)
print("%s seconds" % (time.time() - start_time))  #time

501.12113094329834 seconds
[[ 5.81271761e-02  5.81271761e-02  7.10280374e-02 ...  1.11966282e-01
   1.10207073e-01  1.10207073e-01]
 [-9.71445147e-17 -9.71445147e-17  1.38222767e-14 ... -2.93515212e-14
  -1.51822999e-14 -1.51822999e-14]
 [ 2.59383754e-01  2.59383754e-01  2.26890756e-01 ...  2.07002801e-01
   9.38375350e-02  9.38375350e-02]
 ...
 [ 4.66949666e-02  4.66949666e-02  4.70588235e-02 ...  8.57893673e-02
   8.04932282e-02  8.04932282e-02]
 [ 6.74509804e-01  6.74509804e-01  5.92156863e-01 ...  2.23529412e-01
   2.31372549e-01  2.31372549e-01]
 [ 1.00000000e+00  1.00000000e+00  8.98039216e-01 ...  9.80392157e-02
   9.41176471e-02  9.41176471e-02]]
365.04761815071106 seconds


In [ ]:
#save
'''
pickle.dump(kmeans_normal, open("/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/10000/kmeans_normal.pkl", "wb"))
pickle.dump(kmeans_attack, open("/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/10000/kmeans_attack.pkl", "wb"))
np.save('/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/10000/centroids_normal.npy', centroids_normal)
np.save('/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/10000/centroids_attack.npy', centroids_attack) 
'''

In [ ]:
#load 

'''
f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/MAGNETO/12x12/Ytrain.pickle', 'rb')
YTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/MAGNETO/12x12/Ytest.pickle', 'rb')
YTest = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/NLS-KDD/gradcam/pickle/testing/XTest_heatmap_12x12.pickle', 'rb')
XTest = pickle.load(f_myfile)
f_myfile.close()

XTest = np.array(XTest)
XTest = np.uint8(255 * XTest)
XTest = XTest / 255

XTest = XTest.reshape(-1, (12*12))
'''
#load model 1
'''
kmeans_normal = pickle.load(open("/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/1000/kmeans_normal.pkl", "rb"))
kmeans_attack = pickle.load(open("/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/1000/kmeans_attack.pkl", "rb"))
centroids_normal = kmeans_normal.cluster_centers_
centroids_attack = kmeans_attack.cluster_centers_
'''
#load centroids
'''
centroids_normal = np.load('/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/1000/centroids_normal.npy')
centroids_attack = np.load('/content/drive/MyDrive/ESPERIMENTI/GRACE/NLS-KDD/1000/centroids_attack.npy')
'''

#reshape inverso
'''
img = XTrain[0].reshape(12,12)
plt.imshow(img)
'''
#Stampa centroidi
'''
img = centroids_normal[0].reshape(12,12)
plt.imshow(img)
'''

In [7]:
print(centroids_normal.shape)
print(centroids_attack.shape)

#centroids_normal= np.array(centroids_normal)
#centroids_attack= np.array(centroids_attack)

centroids = np.concatenate((centroids_normal, centroids_attack), axis=0)
centroidsY = np.concatenate((np.ones(1000), np.zeros(1000)), axis=0)

print(centroids.shape)
print(centroidsY.shape)

(1000, 144)
(1000, 144)
(2000, 144)
(2000,)


In [8]:
#RISULTATI kNN (senza PCA)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

start_time = time.time()  #time
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(centroids, centroidsY)
print("%s seconds" % (time.time() - start_time))  #time


start_time = time.time()  #time
y_pred = classifier.predict(XTest)
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time

con_mat = confusion_matrix(YTest, y_pred)

N_CLASSES = 2
tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print('RESULT')
print(r)

print("%s seconds" % (time.time() - start_time))  #time

0.02631521224975586 seconds
2.214939594268799 seconds
RESULT
(10572, 2261, 945, 8766, 0.8577892122072391, 0.8632506396601036, 0.9179473821307632, 0.823813605548196, 0.8683367556468172, 0.09731232622798888, 0.823813605548196)
0.02439093589782715 seconds


# UNSW-NB15

In [ ]:
#import

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
from keras.models import load_model

import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from PIL import Image
import pickle

from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D
from keras.models import Model

from tensorflow.keras import backend as K
#tf.compat.v1.disable_eager_execution()
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D
from sklearn.metrics import confusion_matrix


import re
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.image as mpimg
import keras.backend as K

from keras.preprocessing import image
from keras.layers import Dense, Input, Layer, InputSpec, Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, normalized_mutual_info_score, adjusted_rand_score#, confusion_matrix
from sklearn.utils.linear_assignment_ import linear_assignment

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

from keras import callbacks
from sklearn.model_selection import train_test_split

import time

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/gradcam/pickle/unico/XTrain_heatmap_15x15.pickle', 'rb')
XTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/gradcam/pickle/unico/XTest_heatmap_15x15_1.pickle', 'rb')
XTest1 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/gradcam/pickle/unico/XTest_heatmap_15x15_2.pickle', 'rb')
XTest2 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/gradcam/pickle/unico/XTest_heatmap_15x15_3.pickle', 'rb')
XTest3 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/gradcam/pickle/unico/XTest_heatmap_15x15_4.pickle', 'rb')
XTest4 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/gradcam/pickle/unico/XTest_heatmap_15x15_5.pickle', 'rb')
XTest5 = pickle.load(f_myfile)
f_myfile.close()

XTest = XTest1 + XTest2 + XTest3 + XTest4 + XTest5

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/MAGNETO/15x15/15x15/Ytrain.pickle', 'rb')
YTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/UNSW-NB15/MAGNETO/15x15/15x15/Ytest.pickle', 'rb')
YTest = pickle.load(f_myfile)
f_myfile.close()


XTrain = np.array(XTrain)
XTrain = np.uint8(255 * XTrain)
XTrain = XTrain / 255

XTest = np.array(XTest)
XTest = np.uint8(255 * XTest)
XTest = XTest / 255

XTrain = XTrain.reshape(-1, (15*15))
XTest = XTest.reshape(-1, (15*15))

print(XTrain.shape)
print(XTest.shape)

(82332, 225)
(175341, 225)


In [ ]:
XTrain_normal = XTrain[np.where(YTrain[:] == 1)]
XTrain_attack = XTrain[np.where(YTrain[:] == 0)]

print('Train')
print('Train normal: ', XTrain_normal.shape)
print('Train attack: ', XTrain_attack.shape)

Train
Train normal:  (37000, 225)
Train attack:  (45332, 225)


In [ ]:
#sklearn.cluster.KMeans

from sklearn.cluster import KMeans
import numpy as np
import time

start_time = time.time()  #time
kmeans_normal = KMeans(n_clusters=500, random_state=0).fit(XTrain_normal)
centroids_normal = kmeans_normal.cluster_centers_
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time
kmeans_attack = KMeans(n_clusters=500, random_state=0).fit(XTrain_attack)
centroids_attack = kmeans_attack.cluster_centers_
print("%s seconds" % (time.time() - start_time))  #time

159.21177339553833 seconds
187.47879576683044 seconds


In [ ]:
#save
pickle.dump(kmeans_normal, open("/content/drive/MyDrive/ESPERIMENTI/GRACE/UNSW-NB15/500/kmeans_normal.pkl", "wb"))
pickle.dump(kmeans_attack, open("/content/drive/MyDrive/ESPERIMENTI/GRACE/UNSW-NB15/500/kmeans_attack.pkl", "wb"))
np.save('/content/drive/MyDrive/ESPERIMENTI/GRACE/UNSW-NB15/500/centroids_normal.npy', centroids_normal)
np.save('/content/drive/MyDrive/ESPERIMENTI/GRACE/UNSW-NB15/500/centroids_attack.npy', centroids_attack) 

In [ ]:
print(centroids_normal.shape)
print(centroids_attack.shape)

#centroids_normal= np.array(centroids_normal)
#centroids_attack= np.array(centroids_attack)

centroids = np.concatenate((centroids_normal, centroids_attack), axis=0)
centroidsY = np.concatenate((np.ones(500), np.zeros(500)), axis=0)

print(centroids.shape)
print(centroidsY.shape)

(500, 225)
(500, 225)
(1000, 225)
(1000,)


In [ ]:
#RISULTATI kNN (senza PCA)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

start_time = time.time()  #time
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(centroids, centroidsY)
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time
y_pred = classifier.predict(XTest)
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time

con_mat = confusion_matrix(YTest, y_pred)

N_CLASSES = 2
tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print('RESULT')
print(r)

print("%s seconds" % (time.time() - start_time))  #time

0.01836705207824707 seconds
24.90867257118225 seconds
RESULT
(116939, 2402, 9968, 46032, 0.9294517540107562, 0.900936400734031, 0.921454293301394, 0.9798728014680621, 0.9497660894707775, 0.178, 0.9798728014680621)
0.14067625999450684 seconds


# KDDCUP99

In [ ]:
#import

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
from keras.models import load_model

import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from PIL import Image
import pickle

from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D
from keras.models import Model

from tensorflow.keras import backend as K
#tf.compat.v1.disable_eager_execution()
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D
from sklearn.metrics import confusion_matrix


import re
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.image as mpimg
import keras.backend as K

from keras.preprocessing import image
from keras.layers import Dense, Input, Layer, InputSpec, Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, normalized_mutual_info_score, adjusted_rand_score#, confusion_matrix
from sklearn.utils.linear_assignment_ import linear_assignment

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

from keras import callbacks
from sklearn.model_selection import train_test_split

import time

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#HEATMAP

from __future__ import print_function
import time
import numpy as np
import pandas as pd
#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import time

#heatmap-XTrain(11)

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_1.pickle', 'rb')
XTrain1 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_2.pickle', 'rb')
XTrain2 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_3.pickle', 'rb')
XTrain3 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_4.pickle', 'rb')
XTrain4 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_5.pickle', 'rb')
XTrain5 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_6.pickle', 'rb')
XTrain6 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_7.pickle', 'rb')
XTrain7 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_8.pickle', 'rb')
XTrain8 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_9.pickle', 'rb')
XTrain9 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_10.pickle', 'rb')
XTrain10 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTrain_heatmap_13x13_11.pickle', 'rb')
XTrain11 = pickle.load(f_myfile)
f_myfile.close()

#heatmap-XTest(8)

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_1.pickle', 'rb')
XTest1 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_2.pickle', 'rb')
XTest2 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_3.pickle', 'rb')
XTest3 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_4.pickle', 'rb')
XTest4 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_5.pickle', 'rb')
XTest5 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_6.pickle', 'rb')
XTest6 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_7.pickle', 'rb')
XTest7 = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/gradcam/pickle/validation/XTest_heatmap_13x13_8.pickle', 'rb')
XTest8 = pickle.load(f_myfile)
f_myfile.close()


XTrain = np.concatenate((XTrain1, XTrain2, XTrain3, XTrain4, XTrain5, XTrain6, XTrain7, XTrain8, XTrain9, XTrain10), axis=0)
XTest = np.concatenate((XTest1, XTest2, XTest3, XTest4, XTest5, XTest6, XTest7), axis=0)

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/MAGNETO/13x13/Ytrain.pickle', 'rb')
YTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open('/content/drive/MyDrive/ESPERIMENTI/KDDCUP99/MAGNETO/13x13/Ytest.pickle', 'rb')
YTest = pickle.load(f_myfile)
f_myfile.close()

XTrain = np.array(XTrain)
XTest = np.array(XTest)

XTrain = np.uint8(255 * XTrain)
XTrain = XTrain / 255
XTest = np.uint8(255 * XTest)
XTest = XTest / 255

XTrain = XTrain.reshape(-1, (13*13))
XTest = XTest.reshape(-1, (13*13))

print(XTrain.shape) 
print(YTrain.shape)
print(XTest.shape) 
print(YTest.shape)

(494021, 169)
(494021,)
(311029, 169)
(311029,)


In [ ]:
XTrain_normal = XTrain[np.where(YTrain[:] == 1)]
XTrain_attack = XTrain[np.where(YTrain[:] == 0)]

print('Train')
print('Train normal: ', XTrain_normal.shape)
print('Train attack: ', XTrain_attack.shape)

Train
Train normal:  (97278, 169)
Train attack:  (396743, 169)


In [ ]:
#sklearn.cluster.KMeans

from sklearn.cluster import KMeans
import numpy as np
import time

start_time = time.time()  #time
kmeans_normal = KMeans(n_clusters=500, random_state=0).fit(XTrain_normal)
centroids_normal = kmeans_normal.cluster_centers_
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time
kmeans_attack = KMeans(n_clusters=500, random_state=0).fit(XTrain_attack)
centroids_attack = kmeans_attack.cluster_centers_
print("%s seconds" % (time.time() - start_time))  #time

In [ ]:
#save
pickle.dump(kmeans_normal, open("/content/drive/MyDrive/ESPERIMENTI/GRACE/KDD-CUP99/10000_mb/kmeans_normal.pkl", "wb"))
pickle.dump(kmeans_attack, open("/content/drive/MyDrive/ESPERIMENTI/GRACE/KDD-CUP99/10000_mb/kmeans_attack.pkl", "wb"))
np.save('/content/drive/MyDrive/ESPERIMENTI/GRACE/KDD-CUP99/10000_mb/centroids_normal.npy', centroids_normal)
np.save('/content/drive/MyDrive/ESPERIMENTI/GRACE/KDD-CUP99/10000_mb/centroids_attack.npy', centroids_attack) 

In [ ]:
#****MiniBatchKMeans*******

from sklearn.cluster import MiniBatchKMeans
import numpy as np
import time

start_time = time.time()  #time

kmeans_normal = MiniBatchKMeans(n_clusters=10000, random_state=0).fit(XTrain_normal)
centroids_normal = kmeans_normal.cluster_centers_

print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time

kmeans_attack = MiniBatchKMeans(n_clusters=10000, random_state=0).fit(XTrain_attack)
centroids_attack = kmeans_attack.cluster_centers_
print("%s seconds" % (time.time() - start_time))  #time

357.37306904792786 seconds
399.6844871044159 seconds


In [ ]:
print(centroids_normal.shape)
print(centroids_attack.shape)

#centroids_normal= np.array(centroids_normal)
#centroids_attack= np.array(centroids_attack)

centroids = np.concatenate((centroids_normal, centroids_attack), axis=0)
centroidsY = np.concatenate((np.ones(10000), np.zeros(10000)), axis=0)

print(centroids.shape)
print(centroidsY.shape)

(10000, 169)
(10000, 169)
(20000, 169)
(20000,)


In [ ]:
#RISULTATI kNN (senza PCA)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

start_time = time.time()  #time
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(centroids, centroidsY)
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time
y_pred = classifier.predict(XTest)
print("%s seconds" % (time.time() - start_time))  #time

start_time = time.time()  #time

con_mat = confusion_matrix(YTest, y_pred)

N_CLASSES = 2
tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print('RESULT')
print(r)

print("%s seconds" % (time.time() - start_time))  #time

0.43520426750183105 seconds
44.27198839187622 seconds
RESULT
(227115, 23321, 337, 60256, 0.9239363532017915, 0.9506583527586384, 0.9985183687107609, 0.9068784040633135, 0.9504946765769384, 0.005561698546036671, 0.9068784040633135)
0.24316954612731934 seconds


In [ ]:
print(
904.2282316684723 +
15743.142695727400000 +
25906.5752635002136 +
709.9252226352692 +
2145.7426924705505 +
0.024927616119384766 
)

print(
194.16853904724121 +
1214.857304272620000 +
16310.2735204696655 +
25.797962427139282
)

45409.63903361803
17745.097326216666
